In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
import time

Cities available for PG/Coliving : Noida Delhi Gurgaon Bangalore Indore Pune Chennai Mumbai Hyderabad Ahmedabad

Cities available for BHK/Rooms : Noida Ghaziabad Gurgaon Delhi Bangalore Chennai Mumbai Indore Nagpur

BHK/ROoms : https://www.roomsoom.com/flat/property-listing?lat=&lang=&city=Noida&type=flat

PG/Coliving : https://www.roomsoom.com/pg/property-listing?lat=&lang=&city=Noida&type=pg

In [2]:
# pg/co-living
city_pg = ['Noida', 'Delhi', 'Gurgaon', 'Bangalore', 'Indore', 'Pune', 'Chennai', 'Mumbai', 'Hyderabad', 'Ahmedabad']

# bhk/rooms
city_room = ['Noida', 'Ghaziabad', 'Gurgaon', 'Delhi', 'Bangalore', 'Chennai', 'Mumbai', 'Indore', 'Nagpur']

In [13]:
information = []

Book Now Schedule Visit

Information present in each room: Image Link Rent for 1 --> class name --> Girls/ Boys/ Family BHK Area Rent/MOnth Security Deposit More Details --> Goes to new webpage

In [4]:
def bhk(city):
    html_text = requests.get("https://www.roomsoom.com/flat/property-listing?lat=&lang=&city=" + city +
                             "&type=flat").text
    soup = BeautifulSoup(html_text, 'lxml')
    rooms = soup.find_all('div', class_ = "property_data")
    for room in rooms:
        image_link = room.img['src']
        people_allowed = room.find('div', class_ = "rent_for1").text
        bhk = room.find('div', class_ = "property_title").h3.text
        primitive_location = room.find('div', class_ = "property_title").p.text
        money_info = room.find_all('span', attrs={"class" : "pro_price"})
        rent_per_month = money_info[0].text
        security_deposit = money_info[1].text
        more_details_link = room.find('div', class_ = "col-md-2 col-xs-12").a['href']
        link = requests.get(more_details_link).text
        soup2 = BeautifulSoup(link, 'lxml')
        images1 = soup2.find('ul', class_ = "slides")
        images = []
        for image in images1.find_all('img'):
            images.append(image.get('src'))
        location = soup2.find('ul', class_ = "bed_details_list").li.text.split('|')[0]
        description1 = soup2.find('div', class_="col-md-6 col-sm-12 col-xs-12 border-right").ul
        description = []
        for desc in description1.find_all('li'):
            description.append(desc.text)
        description2 = soup2.find('div', class_="col-md-6 col-sm-12 col-xs-12").ul
        for desc in description2.find_all('li'):
            description.append(desc.text)
        amenities = soup2.find('div', attrs={"id" : "home_amenities"})
        amenities2 = amenities.find('div', class_ = "tab-content").text.replace('\n\n\n', '\n')
        amen = []
        amen = [y for y in (x.strip() for x in amenities2.splitlines()) if y]
        details = soup2.find('div', class_="rent_data").text.replace('\n', '')
        room_item = {
            "city": city,
            "living_type": "bhk/rooms",
            "image_front": image_link,
            "people": people_allowed,
            "bhk": bhk,
            "primitive_location": primitive_location,
            "rent_per_month": rent_per_month,
            "security_deposit": security_deposit,
            "more_details_link": more_details_link,
            "images_links": images,
            "description": description,
            "amenities": amen,
            "details": details,
        }
        information.append(room_item)

In [14]:
def pg(city):
    html_text = requests.get("https://www.roomsoom.com/pg/property-listing?lat=&lang=&city=" + city +
                             "&type=pg").text
    soup = BeautifulSoup(html_text, 'lxml')
    rooms = soup.find_all('div', class_ = "property_data")
    for room in rooms:
        image_link = room.img['src']
        people_allowed = room.find('div', class_ = "rent_for1").text
        name = room.find('div', class_ = "property_title").h3.text
        primitive_location = room.find('div', class_ = "property_title").p.text
        money_info = room.find_all('span', attrs={"class" : "pro_price"})
        rent_per_month = money_info[0].text
        security_deposit = money_info[1].text
        more_details_link = room.find('div', class_ = "col-md-2 col-xs-12").a['href']
        link = requests.get(more_details_link).text
        soup2 = BeautifulSoup(link, 'lxml')
        images1 = soup2.find('ul', class_ = "slides")
        images = []
        for image in images1.find_all('img'):
            images.append(image.get('src'))
        location = soup2.find('ul', class_ = "bed_details_list").li.text.split('|')[0]
        number_of_beds = soup2.find('ul', class_ = "bed_details_list").li.text.split('|')[-1]
        description = []
        description1 = soup2.find('div', class_="col-md-4 border-right col-xs-6").ul
        description2 = soup2.find('div', class_="col-md-4 col-xs-6").ul
        result = "\n".join([" ".join(elem) for elem in zip(description1.text.split("\n"), description2.text.split("\n"))])
        result = result.replace('\n', '\n')
        result = ','.join(result.splitlines())
        result = result[2:-2]
        description.append(result)
        amenities = soup2.find('div', attrs={"id" : "home_amenities"})
        amenities2 = amenities.find('div', class_ = "tab-content").text.replace('\n\n\n', '\n')
        amen = []
        amen = [y for y in (x.strip() for x in amenities2.splitlines()) if y]
        details = soup2.find('div', class_="rent_data").text.replace('\n', '')
        room_item = {
            "city": city,
            "living_type": "pg/co-living",
            "image_front": image_link,
            "people": people_allowed,
            "name": name,
            "primitive_location": primitive_location,
            "rent_per_month": rent_per_month,
            "security_deposit": security_deposit,
            "total_beds": number_of_beds,
            "more_details_link": more_details_link,
            "images_links": images,
            "description": description,
            "amenities": amen,
            "details": details,
        }
        information.append(room_item)

In [15]:
# for city in city_room:
#     bhk(city)

for city in city_pg:
    pg(city)

In [16]:
len(information)

39

In [8]:
pd.DataFrame(information)

,city,living_type,image_front,people,name,primitive_location,rent_per_month,security_deposit,total_beds,more_details_link,images_links,description,amenities,details
0,Noida,pg/co-living,https://www.roomsoom.com/public/image-2020/pro...,boys,RSMNOI1060 Boys PG,"in Sector-15, Noida","₹ 6,000",1 Month Rent,Total Beds : 500,https://www.roomsoom.com/pg-near-Sector-15-in-...,[https://www.roomsoom.com/public/image-2020/pr...,"[Single Sharing 11,000 / Bed,Double Sharing ...","[Bed, Mattress, Pillow & Cover, Bed Sheet, TV,...",This Boys PG in Sector 15 Noida is available f...
1,Noida,pg/co-living,https://www.roomsoom.com/public/image-2020/pro...,girls,RSMNOI1059 Girls PG,"in Sector 15, Noida","₹ 7,000",1 Month Rent,Total Beds : 50,https://www.roomsoom.com/pg-near-Sector-15-in-...,[https://www.roomsoom.com/public/image-2020/pr...,"[Single Sharing 9,500 / Bed,Double Sharing 7...","[Bed, Mattress, Pillow & Cover, Bed Sheet, TV,...",Searching for a Girls PG in Sector 15 Noida? T...
2,Noida,pg/co-living,https://www.roomsoom.com/public/image-2020/pro...,girls,RSMNOI1069 Girls PG,"in Sector 15, Noida","₹ 6,500",1 Month Rent,Total Beds : 50,https://www.roomsoom.com/pg-near-Sector-15-in-...,[https://www.roomsoom.com/public/image-2020/pr...,"[Single Sharing 11,000 / Bed,Double Sharing ...","[Bed, Mattress, Pillow & Cover, Bed Sheet, TV,...",Looking for a Girls PG in Sector 15 Noida? Thi...
3,Noida,pg/co-living,https://www.roomsoom.com/public/image-2020/pro...,boys,RSMNOI1070 Boys PG,"in Sector 15, Noida","₹ 7,000",1 Month Rent,Total Beds : 50,https://www.roomsoom.com/pg-near-Sector-15-in-...,[https://www.roomsoom.com/public/image-2020/pr...,"[Double Sharing 8,000 / Bed,Triple Sharing 7...","[Bed, Mattress, Pillow & Cover, Bed Sheet, TV,...","If you need a Boys PG in Sector 15 Noida, then..."
4,Delhi,pg/co-living,https://www.roomsoom.com/public/image-2020/pro...,boys,RSMTDEL1010 Boys PG,"in Laxmi Nagar, Delhi","₹ 7,000",1 Month Rent,Total Beds : 50,https://www.roomsoom.com/pg-near-Laxmi-Nagar-i...,[https://www.roomsoom.com/public/image-2020/pr...,"[Single Sharing 10,000 / Bed,Double Sharing ...","[Bed, Mattress, Pillow & Cover, Bed Sheet, TV,...","If you need a Boys PG in Laxmi Nagar, Delhi, t..."
5,Delhi,pg/co-living,https://www.roomsoom.com/public/image-2020/pro...,boys,RSMTDEL1009 Boys PG,"in Rohini, Delhi","₹ 7,000",1 Month Rent,Total Beds : 50,https://www.roomsoom.com/pg-near-Rohini-in-Del...,[https://www.roomsoom.com/public/image-2020/pr...,"[Double Sharing 7,000 / Bed,Triple Sharing 6...","[Bed, Mattress, Pillow & Cover, Bed Sheet, TV,...","If you are looking for a Boys PG in Rohini, De..."
6,Delhi,pg/co-living,https://www.roomsoom.com/public/image-2020/pro...,"girls,family",RSMDEL1051 Girls PG,"in LAJPAT NAGAR, Delhi","₹ 7,000",1 Month Rent,Total Beds : 50,https://www.roomsoom.com/pg-near-LAJPAT-NAGAR-...,[https://www.roomsoom.com/public/image-2020/pr...,"[Double Sharing 6,500 / Bed,Triple Sharing 5...","[Bed, Mattress, Pillow & Cover, Bed Sheet, TV,...","Need a Girls PG in Lajpat Nagar, Delhi? This a..."
7,Delhi,pg/co-living,https://www.roomsoom.com/public/image-2020/pro...,boys,RSMTDEL1011 Boys PG,"in Mukharji Nagar , Delhi","₹ 7,000",1 Month Rent,Total Beds : 50,https://www.roomsoom.com/pg-near-Mukharji-Naga...,[https://www.roomsoom.com/public/image-2020/pr...,"[Single Sharing 10,000 / Bed,Double Sharing ...","[Bed, Mattress, Pillow & Cover, Bed Sheet, TV,...",Suitable for students and working professional...
8,Gurgaon,pg/co-living,https://www.roomsoom.com/public/image-2020/pro...,boys,RSMTGUR1001 Boys PG,"in Sector 14 , Gurgaon","₹ 7,000",1 Month Rent,Total Beds : 50,https://www.roomsoom.com/pg-near-Sector-14--in...,[https://www.roomsoom.com/public/image-2020/pr...,"[Single Sharing 10,500 / Bed,Double Sharing ...","[Bed, Mattress, Pillow & Cover, Bed Sheet, TV,...",This Boys PG in Sector 14 Gurgaon is offered b...
9,Gurgaon,pg/co-living,https://www.roomsoom.com/public/image-2020/pro...,girls,RSMTGUR1002 Girls PG,"in Dlf Phase 1, Gurga